In [ ]:
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter

In [ ]:
code = """
var vscode = require('vscode');
/**
 * @param {vscode.ExtensionContext} context
 */
function activate(context) {
	let editor = vscode.window.activeTextEditor;

	let generate = vscode.commands.registerCommand('example.helloWorld', async function () {

		var context = "";

		if (!editor) {
			editor = vscode.window.activeNotebookEditor;
			if (!editor) {
				// No active editor found
				vscode.window.showErrorMessage('No active text or notebook editor found.');
				context = " ";
			}
		}

		try {
			if (editor.document) {
				// Handling the text editor
				var position = editor.selection.active;
				var currentLine = position.line;
				var start = new vscode.Position(0, 0);
				var end = new vscode.Position(currentLine, editor.document.lineAt(currentLine).text.length);
				var selection = new vscode.Selection(start, end);
				var context = editor.document.getText(selection);

				// Fallback to entire document if the selection is empty
				if (!context.trim()) {
					context = editor.document.getText();
				}

			} else if (editor.notebook) {
				// Handling the notebook editor
				var notebook = editor.notebook;
				var activeCell = editor.selection.active;

				if (!activeCell) {
					vscode.window.showErrorMessage('No active cell found in the notebook editor.');
					context = "";
				}

				// Fallback to fetching all cell contents
				var code = activeCell.document.getText();
				if (!code.trim()) {
					let allCode = '';
					for (var cell of notebook.cells) {
						allCode += cell.document.getText() + '\n';
					}
					return allCode.trim();
				}

				context = code;
			}
		} catch (error) {
			// Fallback to entire document or notebook in case of an error
			vscode.window.showErrorMessage(`Error fetching selected code: ${error.message}`);
			if (editor.document) {
				return editor.document.getText();
			} else if (editor.notebook) {
				let allCode = '';
				for (var cell of editor.notebook.cells) {
					allCode += cell.document.getText() + '\n';
				}
				context = allCode.trim();
			}
		}
		
		//Selecting context code:
		var position = editor.selection.active;
		var currentLine = position.line;
		var start = new vscode.Position(0, 0);
		var end = new vscode.Position(currentLine, editor.document.lineAt(currentLine).text.length);
		var selection = new vscode.Selection(start, end);
		var code = editor.document.getText(selection);

		//Getting task
		var task = await vscode.window.showInputBox({
			placeHolder: 'Enter your value here...', // Placeholder text
			prompt: 'This is a prompt for the input box.', // Optional prompt message
			value: 'Enter your task', // Optional pre-filled value
		  });
		
		if(task){

			
			if(task.trim() != "" && task.trim() != "."){
				
				vscode.window.showInformationMessage('Fetching Code!');
				fetch("https://web-production-ecbc.up.railway.app/generate", {
					method: "POST",
					body: JSON.stringify({
						query: task,
						current_code: code
					}),
					headers: {
						"Content-type": "application/json"
					}
				})
				.then((response) => {
					if (!response.ok) {
						throw new Error('Network response was not ok');
					}
					return response.json(); // Parse response body as JSON
				})
				.then((json) => {
					if (editor.document) {
						var position = editor.selection.active;
						var currentLine = position.line;
						var newPosition = new vscode.Position(currentLine + 1, 0);
						
						editor.edit(editBuilder => {
							// console.log(json.value)
							editBuilder.insert(newPosition, json.value);
						}).then(success => {
							if (!success) {
								vscode.window.showErrorMessage('Failed to insert text into the editor.');
							}
						});
					} else if (editor.notebook) {
						var activeCell = editor.selection.active;
						if (activeCell) {
							var position = activeCell.document.positionAt(activeCell.document.getText().length);
							activeCell.document.insert(position, json.value).then(success => {
								if (!success) {
									vscode.window.showErrorMessage('Failed to insert text into the notebook editor.');
								}
							});
						}
						// editor.edit(editbuilder => editbuilder.insert(new vscode.Position(currentLine+1, 0), json.value))
						// editor.edit(builder => builder.replace(newSelection, json.value))
					}})
					.catch((error) => {
						console.error('Error during fetch:', error);
						vscode.window.showInformationMessage('Error during fetch: ' + error.message);
					});
					
				}
				else{
					vscode.window.showErrorMessage("Task cannot be empty!");
				}
			}
			});
			
			// console.log('Congratulations, your extension is now active!');
			
	let store = vscode.commands.registerCommand('example.store', async function() {

		// let currentWorkspace = vscode.workspace.workspaceFolders[0]; // Assuming there's at least one workspace folder
		// let currentDirectoryPath = currentWorkspace.uri.fsPath;

		const selection = editor.selection;
		let storing_code = editor.document.getText(selection);
		// console.log('Current code:', storing_code);
		vscode.window.showInformationMessage("Updating glyph! This might take some time.");

		fetch("https://web-production-ecbc.up.railway.app/store", {
			method: "POST",
			body: JSON.stringify({
				code: storing_code 
			}),
			headers: {
				"Content-type": "application/json"
			}
			})
			.then((response) => {
				if (!response.ok) {
					throw new Error('Network response was not ok');
				}
				return response.json(); // Parse response body as JSON
			})
			.then((json) => {
				// editor.edit(builder => builder.replace(newSelection, json.value))
				vscode.window.showInformationMessage("glyph updated!"); // Show response as information message
			})
			.catch((error) => {
				// console.error('Error during fetch:', error);
				vscode.window.showInformationMessage('Error during fetch: Try restarting!');
		});
	})



	context.subscriptions.push(generate, store);
}
function deactivate() {}

module.exports = {
	activate,
	deactivate
}
"""

In [ ]:
js_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.JS,
    chunk_size = 2000,
    chunk_overlap = 0
)

docs = js_splitter.create_documents([code])

In [ ]:
docs[1]

In [37]:
from ExtractEntities import get_nodes_edge

In [38]:
res_dict = {'nodes': [], 'edges': []}

In [50]:
kg = KnowledgeGraph()

for doc in docs:
    res = get_nodes_edge(doc.page_content)
    kg.load_from_dict(res)

In [42]:
from KnowledgeGraph import KnowledgeGraph

In [51]:
kg.visualize()

graph.html


In [52]:
kg.save()

In [49]:
docs[0].page_content

"var vscode = require('vscode');\n/**\n * @param {vscode.ExtensionContext} context\n */"